In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame
from ipywidgets import FloatProgress
from time import sleep
from IPython.display import display
import re
import pickle
from urlextract import URLExtract

In [2]:
product_link=[]
c = 0
while (c < 580):

    result=requests.get(f'https://www.marksandspencer.in/on/demandware.store/Sites-mnsindia-Site/en_IN/Search-UpdateGrid?cgid=IN_SubCategory_25097048&prefn1=isOnline&prefv1=online&start={c}&sz=48&selectedUrl=https%3A%2F%2Fwww.marksandspencer.in%2Fon%2Fdemandware.store%2FSites-mnsindia-Site%2Fen_IN%2FSearch-UpdateGrid%3Fcgid%3DIN_SubCategory_25097048%26prefn1%3DisOnline%26prefv1%3Donline%26start%3D48%26sz%3D48')
    c=c+48
    get= result.content
    soup = BeautifulSoup(get,"html.parser")
    attr=soup.find_all('a',{'class':'link pagechoice'})
    for i in attr:
        product_link.append(i.attrs['href'])

In [3]:
len(product_link)

622

In [4]:
products=[]
for i in product_link:
    products.append('https://www.marksandspencer.in'+i)

In [5]:
result = requests.get('https://www.marksandspencer.in/printed-frill-neck-long-sleeve-peplum-top/p/P60489067.html')
c = result.content
soup = BeautifulSoup(c,"html.parser")

In [6]:
import json

In [7]:
element = soup.find_all("script",type="application/ld+json")[1]

js = json.loads(element.string) # <- Calling `.string` to get the JSON
print(js)

{'@context': 'http://schema.org', '@type': 'Product', 'url': 'https://www.marksandspencer.in/printed-frill-neck-long-sleeve-peplum-top/p/P60489067.html', 'name': 'Printed Frill Neck Long Sleeve Peplum Top', 'image': 'https://asset1.cxnmarksandspencer.com/is/image/mands/SD_01_T41_3529R_Y8_X_EC_0', 'description': 'Add a romantic feel to your outfit with this pretty long-sleeved peplum top. Regular fit, with front button fastening. The sweet star print and frilled neck are complemented by a dainty ladder lace trim at the bodice, back and across the shoulders. M&amp;S Collection: easy-to-wear wardrobe staples that combine classic and contemporary styles.', 'sku': 'T41/3529R', 'brand': 'M&amp;S Collection IN', 'offers': {'@type': 'Offer', 'availability': 'http://schema.org/InStock', 'itemCondition': 'NewCondition', 'url': 'https://www.marksandspencer.in/printed-frill-neck-long-sleeve-peplum-top/p/P60489067.html', 'price': 2499, 'priceCurrency': 'INR'}}


In [8]:
#js['brand']
js['url']
#js['offers']['availability']
#"₹"+str(js['offers']['price'])
#js['description']
#js['name']
#js['image']

'https://www.marksandspencer.in/printed-frill-neck-long-sleeve-peplum-top/p/P60489067.html'

In [9]:
#soup.prettify()

In [10]:
#list(soup.children)

In [11]:
size=soup.find('select',{'class':'custom-select form-control select-size'}).text.lstrip().rstrip().split("\n\n\n")[1:]

In [12]:
size

['6 Low in Stock',
 '8 Low in Stock',
 '10',
 '12',
 '14',
 '16',
 '18',
 '20',
 '22 Out of Stock',
 '24 Out of Stock']

In [17]:
a=[]
for i in size:
    if(len(i.split())==1 or i.split()[1]!='Out'):
        a.append(i.split()[0])

In [18]:
a

['6', '8', '10', '12', '14', '16', '18', '20']

In [19]:
s=soup.find('div',{'class':'primary-images col-12 col-lg-6'})

In [20]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
#Gender
Description=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [21]:
for i in products:
    try:
        result = requests.get(i)
        c = result.content
        soup = BeautifulSoup(c,"html.parser")
        element = soup.find_all("script",type="application/ld+json")[1]
        js = json.loads(element.string)
        size=soup.find('select',{'class':'custom-select form-control select-size'})
        size=size.text.lstrip().rstrip().split("\n\n\n")[1:]
    except:
        pass
    
    else:
        Product_Brand.append(js['brand'])
        Product_Link.append(js['url'])
        #js['offers']['availability']
        Price.append("Rs. "+str(js['offers']['price']))
        Description.append(js['description'])
        Product_Name.append(js['name'])
        #Primary_Image_Link.append(js['image'])
        extractor = URLExtract()
        e=soup.find('div',{'id':'productImage'})
        urls = extractor.find_urls(e.attrs['data-product-images'])
        if(len(urls)<=2):
            if(len(urls)==0):
                Primary_Image_Link.append("Not available")
                Secondary_Image_Link.append("Not available")
            else:
                Primary_Image_Link.append(urls[1])
                Secondary_Image_Link.append("Not available")
                
            
        else:
            Primary_Image_Link.append(urls[1])
            Secondary_Image_Link.append(urls[2])
        
        
        s=soup.find('span',{'class':'strike-through list pricecolour'})
        if s==None:
            MRP.append("Rs. "+str(js['offers']['price']))
            #MRP.append("Rs. "+str(s.span.attrs['content'])
        else:
            MRP.append("Rs. "+str(s.span.attrs['content']))  
        s=''
        for i in size:
            if(len(i.split())==1 or i.split()[1]!='Out'):
                s=s+i.split()[0]+" , "
        Size_Available.append(s[:-2])

In [22]:
data=pd.DataFrame({
    "Product":Product_Name,
    "Link":Product_Link,
    "Brand":Product_Brand,
    "Size Available":Size_Available,
    "Selling Price":Price,
    "MRP":MRP,
    "Gender":"Women",
    "Description":Description,
    "Primary_Image":Primary_Image_Link,
    "Secondary_Image":Secondary_Image_Link,
    
})

In [26]:
#len(Product_Link)

343

In [24]:
#data.to_csv("women_m&s.csv",index=False)

In [25]:
#data.shape

(343, 10)

In [27]:
data=pd.read_csv('women_m&s.csv')

In [107]:
s=data['Size Available']

In [109]:
s[0].split(" , ")

['6', '8 Low in Stock', '10', '12', '14', '16', '18 ']

In [120]:
a=[]
for i in range(0,len(s)):
     a.append((str(s[i]).split(' , ')))

In [121]:
Size_Available=[]
for i in a:
    s=''
    for j in i:
        if(len(j.split())==1 or j.split()[1]!='Out'):
            s=s+j.split()[0]+" , "
    Size_Available.append(s[:-2])

In [123]:
len(Size_Available)

623

In [124]:
data['Size Available']=Size_Available

In [126]:
import sqlite3
#from pandas import DataFrame

conn = sqlite3.connect('_all_women_m&s.db')
c = conn.cursor()

c.execute('CREATE TABLE products (Product,Link,Brand,Size Available,Selling Price,MRP,Gender,Description,Primary_Image,Secondary_Image)')
conn.commit()

data.to_sql('products', conn, if_exists='replace', index = False)
conn.close

C:\Users\alokt\anaconda3\lib\site-packages\pandas\core\generic.py:2605: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


<function Connection.close>